In [ ]:
base = 'gs://REDACTED'
network = 'resnet20'
prune = 'global_magnitude_20'
short_or_early = 'early'
version = 'v2'
early_idx = 40
early_delta = 0
train_steps = 71108
batch_size = 50000/128
all_prune_idxs = [0,4701,12514,20326,28139,35951,39858,43764,51576,59389,67201,71108]

In [ ]:
base = 'gs://REDACTED'
network = 'resnet50'
prune = 'prune_global_20'
short_or_early = 'short'
version = 'v10'
early_idx = 9
early_delta = 9
train_steps = 112590
batch_size = 1251
all_prune_idxs = [0, 11259, 22518, 33777, 45036, 56295, 67554, 78813, 90072, 101331, 112590]

In [ ]:
base = 's3://REDACTED-data'
network = 'vgg_16_nofc'
prune = 'prune_global_20_fc'
short_or_early = 'early'
version = 'v2'
early_idx = 40
early_delta = 40
train_steps = 71108
batch_size = 50000/128
all_prune_idxs = [0,4701,12514,20326,28139,35951,39858,43764,51576,59389,67201,71108]

In [ ]:
base = 's3://REDACTED-data'
network = 'vgg_19'
prune = 'prune_global_20_fc'
short_or_early = 'early'
version = 'v3'
early_idx = 40
early_delta = 40
train_steps = 71108
batch_size = 50000/128
all_prune_idxs = [0,4701,12514,20326,28139,35951,39858,43764,51576,59389,67201,71108]

In [ ]:
import os
env = {
    'AWS_REGION': 'us-standard',
    'S3_ENDPOINT': 's3.us.cloud-object-storage.appdomain.cloud',
    'AWS_ACCESS_KEY_ID': 'dcd241d33c7142d6b8a80251e05375fc',
    'AWS_SECRET_ACCESS_KEY': 'd30074e169ee8aa885dba41819c247118e163878f47abc32',
    'S3_USE_HTTPS': '1',
    'S3_VERIFY_SSL': '0',
}
print('\n'.join('export {}={}'.format(k, v) for (k, v) in env.items()))
os.environ.update(env)

In [ ]:
%matplotlib inline

import argparse
import os
import re
import numpy as np
import utils
import tensorflow as tf
import matplotlib.pyplot as plt
import functools

import gfile
type_re = re.compile(r'.*?(finetune/finetune_(?P<ft_idx>\d+)'
                     r'|lottery/prune_(?P<prune_idx>\d+)$'
                     r'|lottery_(early|short)_(?P<early_epochs>\d+)/prune_(?P<prune_early_idx>\d+)$'
                     r'|reinit/prune_(?P<reinit_idx>\d+)$'
                     r')')

In [ ]:
tf.logging.set_verbosity(tf.logging.ERROR)

In [ ]:
utils.iter_datum_of_iter_dir(('gs://REDACTED/results/resnet50/prune_global_20/v10/finetune/finetune_9/trial_1/iter_0', False))

In [ ]:
dirs = ['{base}/results/{network}/{prune}/{version}/lottery/prune_{idx}'.format(
    base=base,
    network=network,
    prune=prune,
    version=version,
    idx=p,
) for p in all_prune_idxs]
dirs += ['{base}/results/{network}/{prune}/{version}/finetune/finetune_{idx}'.format(
    base=base,
    network=network,
    prune=prune,
    version=version,
    idx=int((train_steps - p) / batch_size),
) for p in all_prune_idxs]
dirs += ['{base}/results/{network}/{prune}/{version}/lottery_{short_or_early}_{early_idx}/prune_{idx}'.format(
    base=base,
    network=network,
    prune=prune,
    version=version,
    short_or_early=short_or_early,
    early_idx=early_idx,
    idx=p,
) for p in all_prune_idxs]
dirs += ['{base}/results/{network}/{prune}/{version}/reinit/prune_{idx}'.format(
    base=base,
    network=network,
    prune=prune,
    version=version,
    idx=p,
) for p in all_prune_idxs]

experiments = utils.experiments_of_directories(dirs)

In [ ]:
print(len(experiments))

In [ ]:
for e in experiments:
    for t in e.trial_data:
        t.iter_data[:] = list(filter(lambda x: x is not None, t.iter_data))

In [ ]:
for e in experiments:
    for t in e.trial_data:
        dname, rest = os.path.split(e.experiment_dir)
        
        iter_dir = os.path.join(os.path.dirname(dname), 'lottery', rest, t.trial, 'iter_0')
        res = lambda: utils.iter_datum_of_iter_dir((
            iter_dir,
            False,
        ))
        
        if len(t.iter_data) == 0 or 'finetune' in e.experiment_dir:
            continue
        if t.iter_data[0] is None:
            print('fetching {}'.format(iter_dir))
            t.iter_data[0] = res()
        elif t.iter_data[0].iter != 'iter_0':
            print('fetching {}'.format(iter_dir))
            t.iter_data.insert(0, res())

In [ ]:
def _get_iter(expt, trial, iter_idx):
    if isinstance(trial.iter_data, list):
        iter_data = trial.iter_data
    else:
        iter_data = trial.iter_data.get()

    iter_data = list(filter(lambda x: x is not None, iter_data))

    try:
        return next(i for i in iter_data if i.iter == 'iter_{}'.format(iter_idx))
    except StopIteration:
        return

def get_expt(lottery, eps, early=False, reinit=False):
    for expt in experiments:
        m = type_re.match(expt.experiment_dir.rstrip('/'))

        if m.group('reinit_idx') and reinit and eps == int(train_steps/batch_size - round(int(m.group('reinit_idx')) / batch_size)):
            return expt
        elif m.group('ft_idx') and not lottery and not reinit and eps == int(m.group('ft_idx')):
            return expt
        elif m.group('prune_idx') and lottery and not early and not reinit and eps == int(train_steps/batch_size - round(int(m.group('prune_idx')) / batch_size)):
            return expt
        elif m.group('prune_early_idx') and lottery and early and not reinit and eps == int(train_steps/batch_size - round(int(m.group('prune_early_idx')) / batch_size)):
            return expt
        
def get_deltas(expt, it_from, it_to):
    def _get(it):
        data = {}
        for t in expt.trial_data:
            v = _get_iter(expt, t, it)
            if v is None:
                continue
            data[t.trial] = v
        return data
    
    from_ = _get(it_from)
    to_ = _get(it_to)
    
    res = []
    for k in set(from_.keys()) & set(to_.keys()):
        if to_[k].test_acc is not None and from_[k].test_acc is not None:
            res.append(to_[k].test_acc - from_[k].test_acc)
    return np.array(res)

def suffix_of_number(myDate):
    date_suffix = ["th", "st", "nd", "rd"]

    if myDate % 10 in [1, 2, 3] and myDate not in [11, 12, 13]:
        return date_suffix[myDate % 10]
    else:
        return date_suffix[0]

In [ ]:
@functools.lru_cache(1000)
def _get_substeps(directory):
    execution_data_iter_dir = os.path.join(directory, 'eval')
    execution_data_iter_dir = execution_data_iter_dir.replace('REDACTED/results', 'REDACTED/execution_data')
    events_file = next(x for x in gfile.ListDirectory(execution_data_iter_dir) if x.startswith('events.out'))

    steps_xs = []
    steps_ys = []
    for e in tf.train.summary_iterator(os.path.join(execution_data_iter_dir, events_file)):
        for v in e.summary.value:
            if v.tag == 'accuracy':
                steps_xs.append(e.step)
                steps_ys.append(v.simple_value)

    steps_xs = np.array(steps_xs) - (max(steps_xs) - train_steps)
    steps_ys = np.array(steps_ys)
    return steps_xs, steps_ys

In [ ]:
import multiprocessing

pool = multiprocessing.Pool(5)

def get_substeps_of_expt_iter(expt, iter_idx, eps):
    all_steps_xs = None
    all_steps_ys = []
    
    dirs = [os.path.join(expt.experiment_dir, t.trial, 'iter_{}'.format(iter_idx)) for t in expt.trial_data]
    for steps_xs, steps_ys in pool.map(_get_substeps, dirs):
        if all_steps_xs is not None:
            assert np.allclose(steps_xs, all_steps_xs)
        all_steps_xs = steps_xs
        
        if all_steps_ys is None:
            all_steps_ys = np.zeros_like(steps_ys)
        all_steps_ys.append(steps_ys)
        
    ys =  np.array(all_steps_ys) - np.array([_get_iter(expt, t, 0).test_acc for t in expt.trial_data]).reshape(-1, 1)
    return all_steps_xs / batch_size - (train_steps/batch_size - eps), ys

In [ ]:
import matplotlib
t20 = matplotlib.cm.get_cmap('tab20')

In [ ]:
idxs = list(range(1, 10))
import collections
cumsum_p = collections.defaultdict(float)
cumsum_f = collections.defaultdict(float)

# plt.figure(figsize=(16, 4 * len(idxs)))
# plt.figure(figsize=(8, 4 * len(idxs)))

def get_centers(arr):
    if center_type == 'mean':
        return np.array([np.array(x).mean() for x in arr])
    elif center_type == 'median':
        return np.array([np.median(np.array(x)) for x in arr])
    else:
        raise ValueError('Unkown center type {}'.format(center_type))
    
def get_errors(arr):
    if err_type == 'std':
        return [np.array(x).std() for x in arr]
    elif err_type == 'minmax':
        centers = get_centers(arr)
        mins = np.array([x.min() for x in arr])
        maxs = np.array([x.max() for x in arr])
        return np.stack([
            centers - mins,
            maxs - centers,
        ])
    else:
        raise ValueError('Unknown err type {}'.format(err_type))
        
    
def get_experiments_at_idx(idx):
    prune_expts = {}
    prune_early_expts = {}
    ft_expts = {}
    reinit_expts = {}

    for expt in experiments:
        m = type_re.match(expt.experiment_dir.rstrip('/'))
        deltas = get_deltas(expt, 0, idx)
        
        if len(deltas) == 0:
            continue
            
        if m.group('ft_idx'):
            ft_expts[int(m.group('ft_idx'))] = deltas
        elif m.group('prune_idx'):
            prune_expts[int(train_steps/batch_size - round(int(m.group('prune_idx')) / batch_size))] = deltas
        elif m.group('prune_early_idx'):
            prune_early_expts[int(train_steps/batch_size - round(int(m.group('prune_early_idx')) / batch_size))] = deltas
        elif m.group('reinit_idx'):
            reinit_expts[int(train_steps/batch_size - round(int(m.group('reinit_idx')) / batch_size))] = deltas
    return (prune_expts, prune_early_expts, ft_expts, reinit_expts)

In [ ]:
center_type = 'mean'
err_type = 'std'

fmt_label = lambda x: '{} {} {}'.format(
    x,
    center_type,
    r'$\pm 1$ std' if err_type == 'std' else r'$\pm$ $\min$/$\max$'
)
lot_label = fmt_label('Rewind')
lot_early_label = fmt_label('Rewind & Early Stop')
ft_label = fmt_label('Fine-tune')
reinit_label = fmt_label('Scratch')

In [ ]:
def make_plots(to_plot, save):
    for sp_i, iter_idx in enumerate(idxs):
        print(iter_idx)
        (prune_expts, prune_early_expts, ft_expts, reinit_expts) = get_experiments_at_idx(iter_idx)

        plt.figure(figsize=(8, 4))
        plt.title(r'$\Delta$ test accuracy after {idx}{suffix} pruning iteration (approx. {sparsity:.2%} sparsity)'.format(
            idx=iter_idx,
            suffix=suffix_of_number(iter_idx),
            sparsity=1-.8**iter_idx,
        ))
        plt.ylabel('$\Delta$ in test accuracy')
        plt.xlabel('Re-training Epochs')

        if to_plot['early']:
            lot_early_xs, lot_early_deltas = zip(*sorted(list(prune_early_expts.items())))
            lot_early_xs = lot_early_xs
            lot_early_deltas = lot_early_deltas
            
            plt.errorbar(
                np.array(lot_early_xs) - early_delta, 
                get_centers(lot_early_deltas),
                get_errors(lot_early_deltas),
                fmt='o--', 
                label=lot_early_label,
                color='C0',
            )

        lot_xs, lot_deltas = zip(*sorted(list(prune_expts.items())))
        if to_plot['rewind']:
            plt.errorbar(
                lot_xs, 
                get_centers(lot_deltas),
                get_errors(lot_deltas),
                fmt='o--', 
                label=lot_label,
                color='C1',
            )

        ft_xs, ft_deltas = zip(*sorted(list(ft_expts.items())))
        if to_plot['finetune']:
            plt.errorbar(
                ft_xs, 
                get_centers(ft_deltas),
                get_errors(ft_deltas),
                fmt='o--',
                label=ft_label,
                color='C2',
            )

        if to_plot['scratch'] and reinit_expts:
            reinit_xs, reinit_deltas = zip(*sorted(list(reinit_expts.items())))
            plt.errorbar(
                reinit_xs, 
                get_centers(reinit_deltas),
                get_errors(reinit_deltas),
                fmt='o--',
                label=reinit_label,
                color='C3',
            )


        plt.plot([0, train_steps/batch_size], [0, 0], '--', color=(0,0,0,0.3))
        plt.legend()
        plt.ylim(-0.03, 0.01)
        vals = plt.gca().get_yticks()
        plt.gca().set_yticklabels(['{:+,.2%}'.format(x) for x in vals])
        plt.tight_layout()

        if save:
            with gfile.Open('gs://REDACTED/figures/resnet20/{plots}-sparse-pidx-{pidx}.png'.format(
                plots='-'.join(sorted([k for (k,v) in to_plot.items() if v])),
                pidx=iter_idx
            ), 'wb') as f:
                plt.savefig(f)

In [ ]:
import itertools

idxs = range(30)

for (r,e,f,s) in itertools.product(*([[True, False]]*4)):
    to_plot = {
        'rewind': r,
        'early': e,
        'finetune': f,
        'scratch': s,
    }
    if r and f and e and not (s):
#     if r and not (s or e or f):
        make_plots(to_plot, False)


plt.show()

In [ ]:
[e for e in experiments if 'short' in e.experiment_dir and '67554' in e.experiment_dir][0].experiment_dir

In [ ]:
[e for e in experiments if 'finetune_9' in e.experiment_dir][0].trial_data[-1]